In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 19 17:45:06 2019

@author: Team 3 
"""

import torch
import torch.nn as nn
import math
import torchvision.transforms as transforms
import torchvision as tv
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
CUDA_VISIBLE_DEVICES=(7)
model_path = './model_pth/vgg16_bn-6c64b313.pth'
BATCH_SIZE = 128
LR = 0.001
EPOCH = 5

class VGG(nn.Module):
    def __init__(self, features, num_classes=10): 
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential( 
            #fc6
            nn.Linear(256*1*1, 1024),
            nn.ReLU(),
            nn.Dropout(),
 
            #fc7
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(),
 
            #fc8
            nn.Linear(512, num_classes))
        self._initialize_weights()
 
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
 
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
 
#cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

cfg = {
    'A': [32, 32, 'M', 64, 64, 'M', 128, 128, 128, 'M', 128, 128, 128, 'M', 256, 256, 256, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
} 

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, 3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers) 
 
def vgg16(**kwargs):
    model = VGG(make_layers(cfg['A'], batch_norm=True), **kwargs)
    #model.load_state_dict(torch.load(model_path))
    return model
 
def getData(): 
    transform = transforms.Compose([
        transforms.RandomResizedCrop(48),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.1307],
                             std=[0.3081])])
    trainset = tv.datasets.CIFAR10(
        root='CIFAR10/',  
        train = True , 
        transform = transform,
        download=True,
        )
    testset = tv.datasets.CIFAR10(
        root = 'CIFAR10/', 
        train = False, 
        transform = transform,#tv.transforms.ToTensor(),
        download = True,
        )
    #trainset = tv.datasets.CIFAR10(root='./root/', train=True, transform=transform, download=True)
    #testset = tv.datasets.CIFAR10(root='./root/', train=False, transform=transform, download=True)
 
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader
 
def train():
    trainset_loader, testset_loader = getData()
    net = vgg16().cuda()
    net.train()
 
    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)
    # Train the model
    test_losses = []
    test_acces = []
    losses = []
    acces = []
    flag=0
    for epoch in range(EPOCH):
        for step, (inputs,labels) in enumerate(trainset_loader):
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.cuda()
            labels = labels.cuda()
            output = net(inputs)
            loss = criterion(output, labels)
            losses.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total = labels.size(0)
            _, predicted = torch.max(output.data, 1)
            correct = (predicted == labels).sum().item()
            acces.append(correct / total)
            
            #if step % 300 ==0:
            print('Epoch:', epoch, '|Step:', step,  
              '|train loss:%.4f'%loss.data.item())
            acc,los = test(net, testset_loader)
            print('Epoch', epoch, '|step ', step, 'loss: %.4f' %loss.item(), 'test accuracy:%.4f' %acc)
            test_acces.append(acc)
            test_losses.append(los.item())
            if step == 20:
              flag=1
              break
        if flag==1:
          break
        print('Finished Training')
        
        
    plt.plot(losses)
    plt.plot(test_losses)
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()

    plt.plot(acces)
    plt.plot(test_acces)
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()
    
    return net

def test(net, testdata):
    criterion = nn.CrossEntropyLoss()
    correct, total = .0, .0
    for inputs, labels in testdata:
        net.eval()
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    net.train()
    return float(correct) / total ,loss

if __name__ == '__main__':
    net = train()

Files already downloaded and verified
Files already downloaded and verified
Epoch: 0 |Step: 0 |train loss:2.3045
Epoch 0 |step  0 loss: 2.3045 test accuracy:0.1000
Epoch: 0 |Step: 1 |train loss:2.2983
Epoch 0 |step  1 loss: 2.2983 test accuracy:0.1000
Epoch: 0 |Step: 2 |train loss:2.3095
Epoch 0 |step  2 loss: 2.3095 test accuracy:0.1051
Epoch: 0 |Step: 3 |train loss:2.3324
Epoch 0 |step  3 loss: 2.3324 test accuracy:0.0979
Epoch: 0 |Step: 4 |train loss:2.2860


KeyboardInterrupt: ignored